Ссылка, на источник текста

In [ ]:
#DATA_URL = "http://az.lib.ru/t/tolstoj_a_k/text_0180.shtml" #test
DATA_URL = "http://az.lib.ru/l/leskow_n_s/text_0246.shtml" #Andrew's

Устанавливаем библиотеки

In [ ]:
! pip install -q PyYaml==5.3.1
! pip install -q rnnmorph==0.4.0

Создаём объект морфологического анализатора RNNMorph

In [ ]:
%tensorflow_version 1.x
import warnings
warnings.filterwarnings('ignore')

from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru") #ru

Скачиваем текст, по которому будет дано задание, с помощью urllib

In [ ]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами


Удаляем из текста HTML-теги с помощью простого регулярного выражения (используем их, так как структура документа и небольшие ошибки с извлечением текста нас мало волнуют)

In [ ]:
import re

clean_pattern = re.compile("<.*?>")

def clean_html(raw_html: str):
  clean_text = re.sub(clean_pattern, " ", raw_html)
  return clean_text

cleaned_text = clean_html(raw_text)#Текс без html тегов

С помощью библиотеки NLTK разбиваем текст на предложения и токены.

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


"A total of 468 'sentences'"

Задание 1.
С помощью метода str.isalpha из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [ ]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent if pred.normal_form.isalpha()]
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences")]
#predictions[-12:-11] #Сейчас видно, что токены типа "точка", "запятая", "дефис" и тд пока присутствуют в предложениях. От них нужно избавиться

sentences: 100%|██████████| 468/468 [00:00<00:00, 101140.47it/s]


In [ ]:
len(predictions)

468

In [ ]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 

9951

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [ ]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что']

In [ ]:
freq_words = FreqDist()
for word in non_uniq_tokens:
     freq_words[word.lower()]+=1

sto_freq_words = freq_words.most_common(100)

filtered_sentence = [w for w in freq_words if freq_words[w.lower()] > 8 and not w in stop_words]
sto_freq_words, len(filtered_sentence)/100

([('nbsp', 876),
  ('и', 495),
  ('он', 363),
  ('а', 261),
  ('в', 254),
  ('не', 238),
  ('что', 169),
  ('на', 133),
  ('говорить', 122),
  ('весь', 100),
  ('быть', 93),
  ('государь', 89),
  ('левша', 87),
  ('это', 85),
  ('я', 85),
  ('как', 83),
  ('с', 81),
  ('так', 74),
  ('у', 72),
  ('мы', 68),
  ('плат', 65),
  ('но', 52),
  ('же', 52),
  ('к', 49),
  ('один', 48),
  ('англичанин', 48),
  ('такой', 48),
  ('вы', 48),
  ('свой', 47),
  ('этот', 44),
  ('потому', 42),
  ('только', 42),
  ('отвечать', 42),
  ('тот', 41),
  ('чтобы', 41),
  ('мочь', 41),
  ('за', 38),
  ('который', 37),
  ('из', 37),
  ('то', 36),
  ('сейчас', 36),
  ('ничто', 35),
  ('очень', 34),
  ('по', 32),
  ('мастер', 32),
  ('от', 31),
  ('самый', 31),
  ('блоха', 30),
  ('ещё', 30),
  ('ты', 30),
  ('наш', 30),
  ('себя', 29),
  ('да', 29),
  ('другой', 28),
  ('сказать', 27),
  ('тут', 26),
  ('до', 26),
  ('знать', 25),
  ('какой', 25),
  ('стать', 24),
  ('взять', 23),
  ('потом', 23),
  ('нет', 2

Задание 3.
Вычислите, сколько токенов встречается в тексте строго больше 50 раз.

In [ ]:
tokens_upper_freq = [w for w in freq_words if freq_words[w.lower()] > 10]
len(tokens_upper_freq)

126